In [1]:
import os
from os import path
os.environ['QT_QPA_PLATFORM'] = 'offscreen'

In [2]:
import sys
sys.path.insert(1, '../scripts')

In [38]:
import random
#import numpy as np
import pandas as pd
#from scipy import stats
from itertools import groupby
from operator import itemgetter
#import matplotlib.pyplot as plt
#import seaborn as sns
import parseaf as pa
import mdtraj as md
#import alignment_quality as aq
#import crutil

In [4]:
#from Bio import SeqIO
#from Bio import AlignIO
#from Bio.Seq import Seq
#from Bio import SeqIO
#from Bio.SeqRecord import SeqRecord

In [5]:
#from sklearn import metrics
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression

In [6]:
import warnings
warnings.filterwarnings('ignore')

## Functions

In [8]:
def find_helix_indices(ss, bfactor):
    helix_indices = []
    for i, label in enumerate(ss):
        if (label == 'H') and (bfactor[i] >= 70):
            helix_indices.append(i)
    return helix_indices

In [9]:
def find_disordered_indices(ss, bfactor):
    disordered_indices = []
    for i, label in enumerate(ss):
        if ((label == 'C') and (bfactor[i] >= 70)) or (bfactor[i] < 70):
            disordered_indices.append(i)
    return disordered_indices

In [10]:
def list_to_regions(indices, minlen=25):
    ranges = []
    for k, g in groupby(enumerate(indices), lambda x:x[0]-x[1]):
        group = list(map(itemgetter(1), g))
        if group[-1] - group[0] + 1 >= minlen:
            ranges.append((group[0], group[-1]))
    return ranges

In [11]:
def append_sequence(row):
    fdir = '/mnt/d/research/drummond-lab/data/yeast-alphafold-output-unzipped/'
    fpath = fdir + 'AF-' + str(row['uni_id']) + '-F1-model_v1.pdb'
    seq = pa.read_seq_from_pdb(fpath)
    start = row['start']
    end = row['end']
    return seq[start:(end+1)]

In [12]:
def append_len_region(row):
    return len(row['region_seq'])

## Extracting pure helices & pure disordered regions based on AlphaFold output

In [15]:
df_all_orfs = pd.read_table('../../data/sc_orfs/yeast-all-orfs.txt', header=None, names=['orf'])

In [16]:
all_orfs = df_all_orfs['orf'].tolist()

In [17]:
with open('../../data/uniprot-to-sgdid.txt') as f:
    mappings = {}
    for line in f:
        uniprot = line[95:106].rstrip()
        orf = line[75:95].rstrip()
        mappings[orf] = uniprot

In [18]:
uni_id_with_pdb = []
fdir = '/mnt/d/research/drummond-lab/data/cerevisiae-alphafold-output/'
for orf in all_orfs:
    uni_id = mappings[orf]
    fpath = fdir + 'AF-' + str(uni_id) + '-F1-model_v1.pdb'
    if path.exists(fpath):
        uni_id_with_pdb.append(uni_id)

In [27]:
fdir = '/mnt/d/research/drummond-lab/data/cerevisiae-alphafold-output/'
df = pd.DataFrame(columns={'uni_id', 'start', 'end', 'region_seq'})
for uni_id in uni_id_with_pdb:
    fpath = fdir + 'AF-' + str(uni_id) + '-F1-model_v1.pdb'
    pdb = pa.read_af_output(fdir, uni_id)
    ss = md.compute_dssp(pdb, simplified=True)[0]
    bfactor = pa.read_bfactor_from_pdb(fpath)
    helix_indices = find_helix_indices(ss, bfactor)
    regions = list_to_regions(helix_indices)
    seq = pa.read_seq_from_pdb(fpath)
    for r in regions:
        df = df.append({'uni_id': uni_id, 'start': r[0], 'end': r[1], 'region_seq': seq[r[0]:(r[1]+1)]}, ignore_index=True)

In [28]:
df

,region_seq,start,end,uni_id
0,VQDLKQLLLNVFNTYKLERSLSELIQKIIEDSSQDLVQQYRKF,1143,1185,P39702
1,KEDIEKMVAEAEKFKEEDEKESQRIASKNQLESIAYSLKNTISE,508,551,P10591
2,DIIANNAVEEIDRNLNKITKTLNYLRAREWRNMSTVNSTESRLTWL...,138,208,P39704
3,KQMFLGSLFGVVLGVTVAKISILFMYVGITSMLLCEWLRY,103,142,P18411
4,PASMIFRNLLILEDDLRRQAHEQKILKWQFTLFLASMAGVGAFTFYELY,45,93,P18410
...,...,...,...,...
6877,GVYGSVFYAGTGLHFLHMVMLAAMLGVNYWRMR,198,230,P00420
6878,VGYETTIIYTHVLDVIWLFLYVVFYWW,240,266,P00420
6879,NFSALKNVIRCSIIHEYISKFVEREQD,184,210,P03871
6880,SQRMANLKIRRTKFKSVLYHILKEL,282,306,P03871


In [29]:
df['len_region'] = df.apply(lambda row: append_len_region(row), axis=1)
df['label'] = 'helix'

In [30]:
fdir = '/mnt/d/research/drummond-lab/data/cerevisiae-alphafold-output/'
disdf = pd.DataFrame(columns={'uni_id', 'start', 'end', 'region_seq'})
for uni_id in uni_id_with_pdb:
    fpath = fdir + 'AF-' + str(uni_id) + '-F1-model_v1.pdb'
    pdb = pa.read_af_output(fdir, uni_id)
    ss = md.compute_dssp(pdb, simplified=True)[0]
    bfactor = pa.read_bfactor_from_pdb(fpath)
    disorder_indices = find_disordered_indices(ss, bfactor)
    regions = list_to_regions(disorder_indices)
    seq = pa.read_seq_from_pdb(fpath)
    for r in regions:
        disdf = disdf.append({'uni_id': uni_id, 'start': r[0], 'end': r[1], 'region_seq': seq[r[0]:(r[1]+1)]}, ignore_index=True)

In [31]:
disdf['len_region'] = disdf.apply(lambda row: append_len_region(row), axis=1)
disdf['label'] = 'disordered'

In [32]:
df_all = pd.concat([df, disdf], axis=0)
df_all.head(10)

,region_seq,start,end,uni_id,len_region,label
0,VQDLKQLLLNVFNTYKLERSLSELIQKIIEDSSQDLVQQYRKF,1143,1185,P39702,43,helix
1,KEDIEKMVAEAEKFKEEDEKESQRIASKNQLESIAYSLKNTISE,508,551,P10591,44,helix
2,DIIANNAVEEIDRNLNKITKTLNYLRAREWRNMSTVNSTESRLTWL...,138,208,P39704,71,helix
3,KQMFLGSLFGVVLGVTVAKISILFMYVGITSMLLCEWLRY,103,142,P18411,40,helix
4,PASMIFRNLLILEDDLRRQAHEQKILKWQFTLFLASMAGVGAFTFYELY,45,93,P18410,49,helix
5,WDEKYTDSVRFVSRTIAYCNIYCLKK,159,184,P18410,26,helix
6,AEIREGWEIYRDEFWAREGARRRKQAHE,225,252,P18410,28,helix
7,VHKQKLLNDLTYAFSSSLRKIDEERSTIEKFDNKIN,372,407,P18409,36,helix
8,DKIEQKWQDEQELKKKEKELKRKNDAEAKRLRMEERKRQQMQKKIA...,176,249,P31376,74,helix
9,PIEMEEQRMTALKEITDIEYKFAQLRQKLYDNQLVRLQTELQMCLE,171,216,P31385,46,helix


In [33]:
df_all.to_csv('../../data/af_regions/sc_af_regions_all.csv', index=False)

## Doing it again for fission yeast

In [10]:
fdir = '/mnt/d/research/drummond-lab/data/pombe-alphafold-output/'
directory = os.fsencode(fdir)
df = pd.DataFrame(columns={'uni_id', 'start', 'end', 'region_seq'})
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".pdb"):
        fpath = fdir + filename
        uni_id = filename.split('-')[1]
        pdb = pa.read_af_output(fdir, uni_id)
        ss = md.compute_dssp(pdb, simplified=True)[0]
        bfactor = pa.read_bfactor_from_pdb(fpath)
        helix_indices = find_helix_indices(ss, bfactor)
        regions = list_to_regions(helix_indices)
        seq = pa.read_seq_from_pdb(fpath)
        for r in regions:
            df = df.append({'uni_id': uni_id,
                            'start': r[0],
                            'end': r[1],
                            'region_seq': seq[r[0]:(r[1]+1)]},
                           ignore_index=True)
        continue

In [11]:
df['len_region'] = df.apply(lambda row: append_len_region(row), axis=1)
df['label'] = 'helix'

In [12]:
fdir = '/mnt/d/research/drummond-lab/data/pombe-alphafold-output/'
directory = os.fsencode(fdir)
disdf = pd.DataFrame(columns={'uni_id', 'start', 'end', 'region_seq'})
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".pdb"):
        fpath = fdir + filename
        uni_id = filename.split('-')[1]
        pdb = pa.read_af_output(fdir, uni_id)
        ss = md.compute_dssp(pdb, simplified=True)[0]
        bfactor = pa.read_bfactor_from_pdb(fpath)
        disorder_indices = find_disordered_indices(ss, bfactor)
        regions = list_to_regions(disorder_indices)
        seq = pa.read_seq_from_pdb(fpath)
        for r in regions:
            disdf = disdf.append({'uni_id': uni_id,
                            'start': r[0],
                            'end': r[1],
                            'region_seq': seq[r[0]:(r[1]+1)]},
                            ignore_index=True)
        continue

In [13]:
disdf['len_region'] = disdf.apply(lambda row: append_len_region(row), axis=1)
disdf['label'] = 'disordered'

In [14]:
df_pombe = pd.concat([df, disdf], axis=0)

In [15]:
df_pombe.to_csv('../../data/af_regions/pombe_af_regions.csv', index=False)

## Doing it one more time for homo sapiens

In [13]:
fdir = '/mnt/d/research/drummond-lab/data/human-alphafold-output/'
directory = os.fsencode(fdir)
df = pd.DataFrame(columns={'uni_id', 'start', 'end', 'region_seq'})
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith("F1-model_v1.pdb"):
        fpath = fdir + filename
        uni_id = filename.split('-')[1]
        pdb = pa.read_af_output(fdir, uni_id)
        ss = md.compute_dssp(pdb, simplified=True)[0]
        bfactor = pa.read_bfactor_from_pdb(fpath)
        helix_indices = find_helix_indices(ss, bfactor)
        regions = list_to_regions(helix_indices)
        seq = pa.read_seq_from_pdb(fpath)
        for r in regions:
            df = df.append({'uni_id': uni_id,
                            'start': r[0],
                            'end': r[1],
                            'region_seq': seq[r[0]:(r[1]+1)]},
                           ignore_index=True)
        continue

In [14]:
df['len_region'] = df.apply(lambda row: append_len_region(row), axis=1)
df['label'] = 'helix'

In [15]:
fdir = '/mnt/d/research/drummond-lab/data/human-alphafold-output/'
directory = os.fsencode(fdir)
disdf = pd.DataFrame(columns={'uni_id', 'start', 'end', 'region_seq'})
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith("F1-model_v1.pdb"):
        fpath = fdir + filename
        uni_id = filename.split('-')[1]
        pdb = pa.read_af_output(fdir, uni_id)
        ss = md.compute_dssp(pdb, simplified=True)[0]
        bfactor = pa.read_bfactor_from_pdb(fpath)
        disorder_indices = find_disordered_indices(ss, bfactor)
        regions = list_to_regions(disorder_indices)
        seq = pa.read_seq_from_pdb(fpath)
        for r in regions:
            disdf = disdf.append({'uni_id': uni_id,
                            'start': r[0],
                            'end': r[1],
                            'region_seq': seq[r[0]:(r[1]+1)]},
                            ignore_index=True)
        continue

In [16]:
disdf['len_region'] = disdf.apply(lambda row: append_len_region(row), axis=1)
disdf['label'] = 'disordered'

In [17]:
df_human = pd.concat([df, disdf], axis=0)

In [18]:
df_human.to_csv('../../data/af_regions/hsapiens_af_regions.csv', index=False)

## Extracting random non-highly-charged regions from yeast AlphaFold output

In [19]:
def append_structure_label(row):
    fdir = '/mnt/d/research/drummond-lab/data/cerevisiae-alphafold-output/'
    uni_id = row['uni_id']
    left_bound = row['left.bound']
    right_bound = row['right.bound']
    label = pa.get_structure_label(fdir, uni_id, left_bound, right_bound)
    return label

In [20]:
df_hc = pd.read_csv('../../data/charged_regions/cr_filtered.csv', comment='#')

In [21]:
df_hc['label'] = df_hc.apply(lambda row: append_structure_label(row), axis=1)

In [23]:
df_all_orfs = pd.read_table('../../data/sc_orfs/yeast-all-orfs.txt', header=None, names=['orf'])

In [24]:
all_orfs = df_all_orfs['orf'].tolist()

In [26]:
with open('../../data/misc/uniprot-to-sgdid.txt') as f:
    mappings = {}
    for line in f:
        uniprot = line[95:106].rstrip()
        orf = line[75:95].rstrip()
        mappings[orf] = uniprot

In [27]:
uni_id_with_pdb = []
fdir = '/mnt/d/research/drummond-lab/data/cerevisiae-alphafold-output/'
for orf in all_orfs:
    uni_id = mappings[orf]
    fpath = fdir + 'AF-' + str(uni_id) + '-F1-model_v1.pdb'
    if path.exists(fpath):
        uni_id_with_pdb.append(uni_id)

In [28]:
df_valid_orfs = pd.read_csv('../../data/sc_orfs/verified_orfs_with_msa.csv')

In [29]:
valid_orfs = df_valid_orfs['systematic_name'].to_list()

In [30]:
with open('../../data/misc/uniprot-to-sgdid.txt') as f:
    mappings = {}
    for line in f:
        uniprot = line[95:106].rstrip()
        orf = line[75:95].rstrip()
        mappings[orf] = uniprot

In [31]:
valid_orfs_uniid = []
for i in valid_orfs:
    valid_orfs_uniid.append(mappings[i])

In [32]:
valid_uniid = list(set(uni_id_with_pdb) & set(valid_orfs_uniid))

In [33]:
len(valid_uniid)

4660

In [34]:
def extract_random_region_from_proteome(uni_id_with_pdb, regionlen, label):
    fdir = '/mnt/d/research/drummond-lab/data/cerevisiae-alphafold-output/'
    random_region = None
    cnt = 0
    while random_region is None:
        randomi = random.randrange(len(valid_uniid))
        uni_id = valid_uniid[randomi]
        fpath = fdir + 'AF-' + str(uni_id) + '-F1-model_v1.pdb'
        seq = pa.read_seq_from_pdb(fpath)
        if len(seq) > regionlen:
            starti = random.randrange(len(seq) - regionlen)
            region_label = pa.get_structure_label(fdir, uni_id, starti, starti + regionlen)
            if region_label == label:
                random_region = seq[starti:(starti + regionlen + 1)] 
        cnt += 1
        if cnt >= 500:
            print(regionlen)
    return uni_id, random_region, starti, (starti + regionlen)

In [35]:
df_hc = df_hc[df_hc.label != 'unclassified']

In [36]:
df_hc = df_hc.dropna(how='any')

In [39]:
df_relaxed = pd.DataFrame(columns = ['uni_id', 'seq', 'left_bound', 'right_bound', 'label'])
for i in range(5):
    for index, row in df_hc.iterrows():
        rv = {}
        regionlen = row['region.len']
        label = row['label']
        params = extract_random_region_from_proteome(uni_id_with_pdb, regionlen, label)
        rv['uni_id'] = params[0]
        rv['seq'] = params[1]
        rv['left_bound'] = params[2]
        rv['right_bound'] = params[3]
        rv['label'] = label
        df_relaxed = df_relaxed.append(rv, ignore_index=True)

In [40]:
len(df_relaxed)

3915

In [42]:
df_relaxed.to_csv('../../data/af_regions/random_af_regions_low_thresh.csv', index=False)

## Timetree

In [3]:
from Bio import Phylo
from ete3 import Tree, NodeStyle, TreeStyle

In [4]:
t = Tree('./data/timetree.nwk', format=1)

In [11]:
for n in t.traverse("postorder"):
    if len(n.name) > 4:
        n.name = n.name.replace("_", " ")

In [12]:
ts = TreeStyle()
ts.show_leaf_name = True
ts.scale_length = 300

In [13]:
t.render("./timetree.png", tree_style=ts, w=50, units='mm', dpi=300)

{'nodes': [[0.7750015492125985,
   210.80042138582678,
   6.975013942913386,
   217.00043377952755,
   0,
   None],
  [135.46491950931025,
   75.95015182283464,
   141.66493190301105,
   82.15016421653543,
   1,
   None],
  [289.0755778562992,
   10.850021688976378,
   295.27559025,
   17.050034082677165,
   2,
   None],
  [140.11492880458584,
   141.0502819566929,
   146.31494119828662,
   147.25029435039372,
   3,
   None],
  [194.25017066619336,
   66.65013323228347,
   200.45018305989416,
   72.85014562598425,
   4,
   None],
  [206.47445025223405,
   48.05009605118111,
   212.67446264593485,
   54.25010844488189,
   5,
   None],
  [303.02560574212606,
   35.65007126377953,
   309.22561813582683,
   41.85008365748032,
   6,
   None],
  [303.02560574212606,
   60.45012083858268,
   309.22561813582683,
   66.65013323228347,
   7,
   None],
  [298.37559644685047,
   85.25017041338583,
   304.57560884055124,
   91.45018280708662,
   8,
   None],
  [216.9711147333376,
   215.45043068110